In [ ]:
# Agregado de Horas de carga completa

# CODIGO PARA CREACIÓN DE IMAGENES DE PRODUCCIÓN, FACTOR DE CAPACIDAD Y HORAS DE CARGA COMPLETA EN KML

import rasterio
import matplotlib.pyplot as plt
import simplekml
import tkinter as tk
from tkinter import filedialog, Entry, Label, Button

def agregar_overlay_a_kml(file_path, image_base, title_base, kml, kml_name_base, modelo, altura, text_size, line_width, title_size):
    image_path = f'{image_base}{altura}.jpg'
    title = f'{title_base} - {modelo} @{altura}'
    kml_name = f'{kml_name_base}{altura}'

    with rasterio.open(file_path) as src:
        data = src.read(1)
        transform = src.transform

    fig, ax = plt.subplots(figsize=(34, 24), dpi=600)
    ax.imshow(data, cmap='viridis')

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            value = data[i, j]
            ax.text(j, i, f'{value:.2f}', ha='center', va='center', color='black', fontsize=text_size,
                    bbox=dict(facecolor='white', edgecolor='black', boxstyle='round', linewidth=line_width))

    ax.text(0, -0.4, title, ha='left', va='top', fontsize=title_size, fontweight='bold', color='white', backgroundcolor='black')
    ax.axis('off')
    fig.savefig(image_path, dpi=600, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    upper_left = transform * (0, 0)
    lower_right = transform * (data.shape[1], data.shape[0])

    ground = kml.newgroundoverlay(name=kml_name)
    ground.icon.href = image_path
    ground.latlonbox.north, ground.latlonbox.west = upper_left[1], upper_left[0]
    ground.latlonbox.south, ground.latlonbox.east = lower_right[1], lower_right[0]

def cargar_archivo(tipo, label_var):
    file_path = filedialog.askopenfilename(title=f"Seleccionar archivo {tipo}", filetypes=[("Archivos TIF", "*.tif")])
    label_var.set(file_path)

def procesar_y_guardar_kml(modelo, altura, nombre_parque, path_energia, path_factor, path_horas_carga, text_size, line_width, title_size):
    modelo_filename = modelo.replace('/', '_').replace(' ', '_')
    nombre_parque_filename = nombre_parque.replace('/', '_').replace(' ', '_')

    kml = simplekml.Kml()
    agregar_overlay_a_kml(path_energia, 'map_hd_GWh_', 'Gross Annual Energy Production Map (GWh)', kml, 'Energy_Production_Map_', modelo, altura, text_size, line_width, title_size)
    agregar_overlay_a_kml(path_factor, 'map_hd_CapFac_', 'Gross Annual Capacity-Factor Map', kml, 'Capacity_Factor_Map_', modelo, altura, text_size, line_width, title_size)
    agregar_overlay_a_kml(path_horas_carga, 'map_hd_LoadHours_', 'Full Load Hours Map', kml, 'Full_Load_Hours_Map_', modelo, altura, text_size, line_width, title_size)

    kml.save(f"{modelo_filename}_{altura}_{nombre_parque_filename}.kml")

root = tk.Tk()
root.title("Selector de Archivos TIF y Datos del Aerogenerador")

path_energia = tk.StringVar()
path_factor = tk.StringVar()
path_horas_carga = tk.StringVar()

Label(root, text="Modelo del Aerogenerador y densidad del aire:").pack()
entry_modelo = Entry(root)
entry_modelo.pack()

Label(root, text="Altura del Aerogenerador:").pack()
entry_altura = Entry(root)
entry_altura.pack()

Label(root, text="Nombre del Parque Eólico:").pack()
entry_nombre_parque = Entry(root)
entry_nombre_parque.pack()

Label(root, text="Tamaño de Texto (Valor Numérico):").pack()
entry_text_size = Entry(root)
entry_text_size.insert(0, "2")  # Valor predeterminado
entry_text_size.pack()

Label(root, text="Ancho de Línea (Valor Numérico):").pack()
entry_line_width = Entry(root)
entry_line_width.insert(0, "0.1")  # Valor predeterminado
entry_line_width.pack()

Label(root, text="Tamaño de Texto del Título (Valor Numérico):").pack()
entry_title_size = Entry(root)
entry_title_size.insert(0, "10")  # Valor predeterminado
entry_title_size.pack()

Label(root, text="Archivo de Energía:").pack()
label_energia = Label(root, textvariable=path_energia, relief="sunken")
label_energia.pack()
Button(root, text="Cargar Archivo de Energía", command=lambda: cargar_archivo("Energía", path_energia)).pack()

Label(root, text="Archivo de Factor de Capacidad:").pack()
label_factor = Label(root, textvariable=path_factor, relief="sunken")
label_factor.pack()
Button(root, text="Cargar Archivo de Factor de Capacidad", command=lambda: cargar_archivo("Factor de Capacidad", path_factor)).pack()

Label(root, text="Archivo de Horas de Carga Completa:").pack()
label_horas_carga = Label(root, textvariable=path_horas_carga, relief="sunken")
label_horas_carga.pack()
Button(root, text="Cargar Archivo de Horas de Carga Completa", command=lambda: cargar_archivo("Horas de Carga Completa", path_horas_carga)).pack()

Button(root, text="Procesar y Guardar KML", command=lambda: procesar_y_guardar_kml(
    entry_modelo.get(), 
    entry_altura.get(), 
    entry_nombre_parque.get(), 
    path_energia.get(), 
    path_factor.get(),
    path_horas_carga.get(),
    float(entry_text_size.get()),
    float(entry_line_width.get()),
    float(entry_title_size.get()))).pack()

root.mainloop()
